<a href="https://colab.research.google.com/github/tommyEzreal/study_low_level/blob/main/pytorch/%EB%AA%A8%EB%91%90%EB%A5%BC%EC%9C%84%ED%95%9C%EB%94%A5%EB%9F%AC%EB%8B%9D2/Convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CNN

output size = [ input - filter + (2*padding) / stride ] + 1

In [1]:
import torch
import torch.nn as nn

# torch.nn.Conv2d(input_channel, output_channel, kernel_size, stride, padding, )



conv = nn.Conv2d(1,1,11,stride=4, padding=0)
conv

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))

In [3]:
# conv(input) -> input = Tensor type, (batch, channel, height, width)
inputs = torch.Tensor(1, 1, 227,227)
inputs.shape

torch.Size([1, 1, 227, 227])

In [6]:
out = conv(inputs)
out.shape

torch.Size([1, 1, 55, 55])

In [ ]:
# pooling 

# torch.nn.MaxPool2d(kernel_size, stride, padding)

In [8]:
# build cnn

inputs = torch.Tensor(1,1,28,28)
conv1 = nn.Conv2d(1,5,5)
pool = nn.MaxPool2d(2)

out = conv1(inputs)
out2 = pool(out)

out.size(), out2.size()

(torch.Size([1, 5, 24, 24]), torch.Size([1, 5, 12, 12]))

Mnist with CNN

In [12]:
# 직접 레이어 통과시켜보며 shape 먼저 확인해보기 

inputs = torch.Tensor(1,1,28,28)
inputs.shape

conv1 = nn.Conv2d(1,32,3,padding=1) # in_c = 1, out_c = 32, kernel=3, stride=1, padding=1
pool = nn.MaxPool2d(2)

conv2 = nn.Conv2d(32, 64, 3, padding=1)

conv1, conv2, pool

(Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))

In [14]:
out = conv1(inputs)
print(out.shape)
out = pool(out)
print(out.shape)
out = conv2(out)
print(out.shape)
out = pool(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


In [19]:
out = out.view(out.size(0),-1)
out.shape

torch.Size([1, 3136])

In [21]:
fc = nn.Linear(3136, 10)
out = fc(out)
out.shape

torch.Size([1, 10])

In [1]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)
if device == 'cuda':
  torch.cuda.manual_seed_all(0)

In [3]:
# params 
learning_rate = 0.001
epochs = 15
batch_size = 64

In [4]:
# datasets 
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [5]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last=True)
                                          

In [15]:
len(data_loader) * 64, len(mnist_train)

(59968, 60000)

In [6]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32,64, 3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc = nn.Linear(7*7*64,10,bias=True)

    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0), -1) # view로 펼치기 
    out = self.fc(out)
    return out 



In [8]:
model = CNN().to(device)
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [9]:
# loss and optim 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
# training loop 

total_batch = len(data_loader)

for epoch in range(epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    h = model(X)
    cost = criterion(h, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('Epoch: {} , cost: {}'.format(
      epoch+1, avg_cost
  ))
print("finished")

Epoch: 1 , cost: 0.18528541922569275
Epoch: 2 , cost: 0.05731066316366196
Epoch: 3 , cost: 0.04157356172800064
Epoch: 4 , cost: 0.0331612154841423
Epoch: 5 , cost: 0.02722359076142311
Epoch: 6 , cost: 0.02157476171851158
Epoch: 7 , cost: 0.018177296966314316
Epoch: 8 , cost: 0.014631454832851887
Epoch: 9 , cost: 0.012551045045256615
Epoch: 10 , cost: 0.010767962783575058
Epoch: 11 , cost: 0.008717143908143044
Epoch: 12 , cost: 0.006607390008866787
Epoch: 13 , cost: 0.005862606223672628
Epoch: 14 , cost: 0.0059044333174824715
Epoch: 15 , cost: 0.004462121520191431
finished


In [12]:
# test 

with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1,28,28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  pred = model(X_test)
  cor_pred = torch.argmax(pred,1) == Y_test
  acc = cor_pred.float().mean()
  print("acc:", round(acc.item(),4))

acc: 0.9834


/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
